In [1]:
"""
1. Consider the 3-node electricity network given in class in Week 2. If you use Excel, you
may use the 3nodelp.xlsx file on Canvas. For the questions below, assume that the
generator at Node 1 is a wind generator with zero marginal cost of production and that
the capacity of the generator at Node 2 is 5 MW’s, and that the load at Node 3 is also 5
MW’s.
a) Suppose the available wind generation at Node 1 is 5 MW’s. What is the amount
of wind generation and Node 2 generation in this case? What would be the LMP’s
at each node? How much of the zero cost wind power is curtailed (i.e., not
available) due to the transmission capacity?
b) Suppose now that the wind generation capacity at Node 1 is random and normally
distributed with a mean of 5 MW’s and a standard deviation of 2 MW’s. Generate
30 scenarios of wind generation capacity and find the generation levels at Nodes 1
and 2, total cost for one hour, LMP’s, and the amount of wind curtailment for each
scenario. You may not be able to meet the load in every scenario. In the event that
wind capacity is insufficient to meet the load, value the lost load at a cost of
$1000/MW-hour. To do this, you can add an additional decision variable in the
linear program for lost load on the left-hand side of the Node 3 flow constraint.
c) Using the same scenarios as you generated for (b), increase the capacity of Line 1
transmission to 1.5 MW’s. With this expanded transmission capacity, repeat your
calculations of the generation at Nodes 1 and 2, total cost for one hour, LMP’s, and
the amount of wind curtailment for each scenario.
d) Using your observations above, assuming these conditions occur during 3000 peak
hours in a year, what would be your estimate of the annual value of increasing the
capacity of Line 1 from 1 MW to 1.5 MW and by what fraction does this
transmission increase the overall efficiency of a wind generator investment (i.e.,
increase in the fraction of the available wind capacity that is able to be used)
"""

'\n1. Consider the 3-node electricity network given in class in Week 2. If you use Excel, you\nmay use the 3nodelp.xlsx file on Canvas. For the questions below, assume that the\ngenerator at Node 1 is a wind generator with zero marginal cost of production and that\nthe capacity of the generator at Node 2 is 5 MW’s, and that the load at Node 3 is also 5\nMW’s.\na) Suppose the available wind generation at Node 1 is 5 MW’s. What is the amount\nof wind generation and Node 2 generation in this case? What would be the LMP’s\nat each node? How much of the zero cost wind power is curtailed (i.e., not\navailable) due to the transmission capacity?\nb) Suppose now that the wind generation capacity at Node 1 is random and normally\ndistributed with a mean of 5 MW’s and a standard deviation of 2 MW’s. Generate\n30 scenarios of wind generation capacity and find the generation levels at Nodes 1\nand 2, total cost for one hour, LMP’s, and the amount of wind curtailment for each\nscenario. You may not 

In [85]:
!pip install pandas
!pip install numpy
!pip install pulp

In [83]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pulp

In [84]:
"""

G1 = Generation Node 1 [This is free generation]
G2 = Generation Node 2 [This is costly 40/hour. Max = 5MW]
**G3** = Generation Node 3 [This is demand node = 5MW]


F1 = Power flow on Line 1 (1->2) [
F2 = Power flow on Line 2 (2->3)
F3 = Power flow on Line 3 (3->1)

Theta 2 =  Voltage angle at Node 2
Theta 3 =  Voltage angle at Node 3
Theta 1 = 0 (this is the reference)
"""

'\n\nG1 = Generation Node 1 [This is free generation]\nG2 = Generation Node 2 [This is costly 40/hour. Max = 5MW]\n**G3** = Generation Node 3 [This is demand node = 5MW]\n\n\nF1 = Power flow on Line 1 (1->2) [\nF2 = Power flow on Line 2 (2->3)\nF3 = Power flow on Line 3 (3->1)\n\nTheta 2 =  Voltage angle at Node 2\nTheta 3 =  Voltage angle at Node 3\nTheta 1 = 0 (this is the reference)\n'

In [3]:
"""
Equations

Flow = angle difference

F1 = Theta_1 - Theta_2
F2 = Theta_2 - Theta_3
F3 = Theta_3 - Theta_1

Where Theta_1 = 0

so

F1 = -Theta_2
F2 = Theta_2 - Theta_3
F3= Theta_3


ie.
F2 = -F1 - F3

"""

'\nEquations\n\nFlow = angle difference\n\nF1 = Theta_1 - Theta_2\nF2 = Theta_2 - Theta_3\nF3 = Theta_3 - Theta_1\n\nWhere Theta_1 = 0\n\nso\n\nF1 = -Theta_2\nF2 = Theta_2 - Theta_3\nF3= Theta_3\n\n\nie.\nF2 = -F1 - F3\n\n'

In [4]:
"""
Node 1

G1 is generating power
Power flows to G2 from F1
Power comes in from N3 via F3


G1 = Generation Node 1
F1 = Power flow on Line 1 from Node 1-> Node 2)
F3 = Power flow on Line 3 from Node 3-> Node 1)

KEY EQUATION
G1 - F1 + F3 = 0
"""

'\nNode 1\n\nG1 is generating power\nPower flows to G2 from F1\nPower comes in from N3 via F3\n\n\nG1 = Generation Node 1\nF1 = Power flow on Line 1 from Node 1-> Node 2)\nF3 = Power flow on Line 3 from Node 3-> Node 1)\n\nKEY EQUATION\nG1 - F1 + F3 = 0\n'

In [5]:
"""
Node 2


G2 Generates power
Recieves Power from Node 1 via F1
Sends power to Node 3 via F2


G2 + F1 - F2 = 0

"""

'\nNode 2\n\n\nG2 Generates power\nRecieves Power from Node 1 via F1\nSends power to Node 3 via F2\n\n\nG2 + F1 - F2 = 0\n\n'

In [6]:
"""
Node 3

No Generation
Load = 5MW

Gets power from Node 2 via F2
Sends power to Node 1 via F3
Recieves Power from Node 1 via F1
Sends power to Node 3 via F2


 -F3 + F2 = 5

"""

'\nNode 3\n\nNo Generation\nLoad = 5MW\n\nGets power from Node 2 via F2\nSends power to Node 1 via F3\nRecieves Power from Node 1 via F1\nSends power to Node 3 via F2\n\n\n -F3 + F2 = 5\n\n'

In [7]:
"""
Constrations

G1: No limit
G3: Max 5 MW



All line flows are between

-5 <= F1,F2,F3 <= 5

"""

'\nConstrations\n\nG1: No limit\nG3: Max 5 MW\n\n\n\nAll line flows are between\n\n-5 <= F1,F2,F3 <= 5\n\n'

In [8]:
"""

Objective

Minimize: 0* G1 + 40MW/Hour * G2

"""

'\n\nObjective\n\nMinimize: 0* G1 + 40MW/Hour * G2\n\n'

In [9]:
#1.1
prob = pulp.LpProblem("Three Node Problem", pulp.LpMinimize)

#Decision Variables

G1=  pulp.LpVariable("G1", lowBound=0)

G2= pulp.LpVariable("G2", lowBound=0, upBound=5)

F1 = pulp.LpVariable("Flow1")
F2 = pulp.LpVariable("Flow2")
F3 = pulp.LpVariable("Flow3")

theta2 = pulp.LpVariable("Theta2")
theta3 = pulp.LpVariable("Theta3")
theta1 = 0


prob += 20 * G1 + 40 * G2


prob += F1 == theta1 - theta2
prob += F2 == theta2 - theta3
prob += F3 == theta3 - theta1


prob += G1 - F1 + F3 == 0, "Node1_Balance"
prob += G2 + F1 - F2 == 0, "Node2_Balance"
prob += F2 - F3 == 5, "Node3_Balance" #Load at 5MW

#constraints

prob += F1 >= -1
prob += F1 <= 1

prob += F2 >=  -5
prob += F2 <= 5

prob += F3 >= -5
prob += F3 <= 5


prob.solve()

print("\nLocational Marginal Prices (LMPs):")
for cname, constraint in prob.constraints.items():
    if "Node" in cname:  # filters Node1_Balance, Node2_Balance, Node3_Balance
        print(f"{cname} → LMP = ${constraint.pi:.2f} per MWh")

print("Status:", pulp.LpStatus[prob.status])
print(f"G1 (Wind): {G1.varValue:.4f} MW")
print(f"G2: {G2.varValue:.4f} MW")
print(f"Total Cost: ${pulp.value(prob.objective):.2f}")





Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/a24a4deb5fd04e3cba18b83717567217-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/a24a4deb5fd04e3cba18b83717567217-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 41 RHS
At line 54 BOUNDS
At line 61 ENDATA
Problem MODEL has 12 rows, 7 columns and 21 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-12) rows, 0 (-7) columns and 0 (-21) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 120
After Postsolve, objective 120, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 120 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from 

In [10]:
import numpy as np

In [11]:
np.random.normal()*2+5

3.0749659699829257

In [77]:
#1.2

#mean 5, std =2
costs = {}
lmp = {}
generation_levels = {}
iter = 30
counter = {}
iterations = 0
wind_curtailment = []

for i in range(iter):
    wind_capacity = np.random.normal(loc = 5, scale = 2)
    wind_capacity_max = max(0, wind_capacity)

    prob = pulp.LpProblem("Three Node Problem", pulp.LpMinimize)

    #Decision Variables

    G1=  pulp.LpVariable("G1", lowBound=0, upBound=wind_capacity_max)

    G2= pulp.LpVariable("G2", lowBound=0, upBound=5)

    LL = pulp.LpVariable("LostLoad", lowBound=0)

    F1 = pulp.LpVariable("Flow1")
    F2 = pulp.LpVariable("Flow2")
    F3 = pulp.LpVariable("Flow3")

    theta2 = pulp.LpVariable("Theta2")
    theta3 = pulp.LpVariable("Theta3")
    theta1 = 0

    #main - similar to the excel sum product
    prob += pulp.lpSum([20.0 * G1, 40.0 * G2, 1000.0 * LL]), "TotalCost"

    prob += F1 == theta1 - theta2
    prob += F2 == theta2 - theta3
    prob += F3 == theta3 - theta1


    prob += G1 - F1 + F3 == 0, "Node1_Balance"
    prob += G2 + F1 - F2 == 0, "Node2_Balance"
    prob += F2 - F3 + LL == 5, "Node3_Balance" #Load at 5MW

    #constraints

    prob += F1 >= -1
    prob += F1 <= 1

    prob += F2 >=  -5
    prob += F2 <= 5

    prob += F3 >= -5
    prob += F3 <= 5


    prob.solve()
######################################

    iterations+=1

    for cname, constraint in prob.constraints.items():
        if "Node" in cname:  # filters Node1_Balance, Node2_Balance, Node3_Balance
            counter[cname] = constraint.pi


    generation_levels[iterations] = [G1.varValue, G2.varValue, LL.varValue]

    wind_curtailment.append(max(0,wind_capacity_max-G1.varValue))


    costs[iterations] = (pulp.value(prob.objective))
    lmp[iterations] = (counter)

x = pd.DataFrame(index = range(1,31), columns = ['Costs', 'Wind Curtailment', 'Node_1_Balance', 'Node_2_Balance', 'Node_3_Balance', 'Generation_Level_1', 'Generation_Level_2', 'Lost_Load'])
x['Costs'] = costs
x['Wind Curtailment'] = wind_curtailment
x['Generation_Level_1'] =  [v[0] for v in generation_levels.values()]
x['Generation_Level_2'] = [v[1] for v in generation_levels.values()]
x['Lost_Load'] = [v[2] for v in generation_levels.values()]
x['Node_1_Balance'] = [v["Node1_Balance"] for v in lmp.values()]
x['Node_2_Balance'] = [v["Node2_Balance"] for v in lmp.values()]
x['Node_3_Balance'] = [v["Node3_Balance"] for v in lmp.values()]



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/d13a61d1124549a4b8a538f59a3eeb85-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/d13a61d1124549a4b8a538f59a3eeb85-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 43 RHS
At line 56 BOUNDS
At line 64 ENDATA
Problem MODEL has 12 rows, 8 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-8) rows, 6 (-2) columns and 12 (-10) elements
0  Obj 0 Primal inf 10.399996 (4)
5  Obj 120
Optimal - objective value 120
After Postsolve, objective 120, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 120 - 5 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from norm

In [78]:
display(x.head())

,Costs,Wind Curtailment,Node_1_Balance,Node_2_Balance,Node_3_Balance,Generation_Level_1,Generation_Level_2,Lost_Load
1,120.000000,2.464768,40.0,40.0,40.0,4.000000,1.000000,0.0
2,120.000000,1.212857,40.0,40.0,40.0,4.000000,1.000000,0.0
3,130.051374,0.000000,40.0,40.0,40.0,3.497431,1.502569,0.0
4,120.000000,1.644520,40.0,40.0,40.0,4.000000,1.000000,0.0
5,120.000000,1.120818,40.0,40.0,40.0,4.000000,1.000000,0.0


In [80]:
#1.3

#mean 5, std =2
costs = {}
lmp = {}
generation_levels = {}
iter = 30
counter = {}
iterations = 0
wind_curtailment = []

for i in range(iter):
    wind_capacity = np.random.normal(loc = 5, scale = 2)
    wind_capacity_max = max(0, wind_capacity)

    prob = pulp.LpProblem("Three Node Problem", pulp.LpMinimize)

    #Decision Variables

    G1=  pulp.LpVariable("G1", lowBound=0, upBound=wind_capacity_max)

    G2= pulp.LpVariable("G2", lowBound=0, upBound=5)

    LL = pulp.LpVariable("LostLoad")

    F1 = pulp.LpVariable("Flow1")
    F2 = pulp.LpVariable("Flow2")
    F3 = pulp.LpVariable("Flow3")

    theta2 = pulp.LpVariable("Theta2")
    theta3 = pulp.LpVariable("Theta3")
    theta1 = 0

    #main - similar to the excel sum product
    prob += pulp.lpSum([20.0 * G1, 40.0 * G2, 1000.0 * LL]), "TotalCost"

    prob += F1 == theta1 - theta2
    prob += F2 == theta2 - theta3
    prob += F3 == theta3 - theta1


    prob += G1 - F1 + F3 == 0, "Node1_Balance"
    prob += G2 + F1 - F2 == 0, "Node2_Balance"
    prob += F2 - F3 + LL == 5, "Node3_Balance" #Load at 5MW

    #constraints

    prob += F1 >= -1.5
    prob += F1 <= 1.5

    prob += F2 >=  -5
    prob += F2 <= 5

    prob += F3 >= -5
    prob += F3 <= 5


    prob.solve()
######################################
    iterations+=1

    for cname, constraint in prob.constraints.items():
        if "Node" in cname:  # filters Node1_Balance, Node2_Balance, Node3_Balance
            counter[cname] = constraint.pi


    generation_levels[iterations] = [G1.varValue, G2.varValue, LL.varValue]

    if LL.varValue <= 0:
        prob.objective = 1000


    costs[iterations] = (pulp.value(prob.objective))
    lmp[iterations] = (counter)

    wind_curtailment.append(max(0,wind_capacity_max-G1.varValue))




df = pd.DataFrame(index = range(1,31), columns = ['Costs', 'Wind Curtailment', 'Node_1_Balance', 'Node_2_Balance', 'Node_3_Balance', 'Generation_Level_1', 'Generation_Level_2', 'Lost_Load'])
df['Costs'] = costs
df['Wind Curtailment'] = wind_curtailment
df['Generation_Level_1'] =  [v[0] for v in generation_levels.values()]
df['Generation_Level_2'] = [v[1] for v in generation_levels.values()]
df['Lost_Load'] = [v[2] for v in generation_levels.values()]
df['Node_1_Balance'] = [v["Node1_Balance"] for v in lmp.values()]
df['Node_2_Balance'] = [v["Node2_Balance"] for v in lmp.values()]
df['Node_3_Balance'] = [v["Node3_Balance"] for v in lmp.values()]



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/b7e4d28f3cc048abbc195e0def55800c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/hq/47ww7tn11ss9dm7r1n54_hlr0000gn/T/b7e4d28f3cc048abbc195e0def55800c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 43 RHS
At line 56 BOUNDS
At line 65 ENDATA
Problem MODEL has 12 rows, 8 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-9) rows, 5 (-3) columns and 9 (-13) elements
0  Obj 8200.0001 Primal inf 4.6999971 (3) Dual inf 2000 (2)
3  Obj -4700
Optimal - objective value -4700
After Postsolve, objective -4700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -4700 - 3 iterations time 0.002, Presolve 0.00
Option for 

In [81]:
df

,Costs,Wind Curtailment,Node_1_Balance,Node_2_Balance,Node_3_Balance,Generation_Level_1,Generation_Level_2,Lost_Load
1,1000,6.276040e-01,20.0,40.0,1000.0,5.000000,5.000000,-5.000000
2,1000,1.201641e+00,20.0,40.0,1000.0,5.000000,5.000000,-5.000000
3,1000,1.115593e-09,20.0,40.0,1000.0,2.988170,5.000000,-2.988170
4,1000,1.717371e+00,20.0,40.0,1000.0,5.000000,5.000000,-5.000000
5,1000,4.662086e-08,20.0,40.0,1000.0,2.880572,5.000000,-2.880572
6,1000,1.824761e-08,20.0,40.0,1000.0,4.894213,5.000000,-4.894213
7,1000,0.000000e+00,20.0,40.0,1000.0,2.503941,5.000000,-2.503941
8,1000,0.000000e+00,20.0,40.0,1000.0,4.524675,5.000000,-4.524675
9,1000,1.564622e-01,20.0,40.0,1000.0,5.000000,5.000000,-5.000000
10,1000,6.307486e-01,20.0,40.0,1000.0,5.000000,5.000000,-5.000000


In [82]:
#1.4